In [1]:
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import os
import sys
from time import time
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy
from tempfile import mkdtemp
from shutil import rmtree

from sklearn.compose import make_column_selector
# from sklearn.ensemble import StackingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from torch import nn
from skorch import NeuralNetClassifier

import My_Transformers
from sklearn.preprocessing import StandardScaler
# Global var
TRAIN_ROW_COUNT = 7964
from My_Transformers import *
from Word2Vec_Transformer import Text_cleaner_transformer
import warnings
warnings.filterwarnings ("ignore")

# Global vars
VERSION = 7.1
trainDF = None
testDF  = None
sclf_1_1, sclf_1_2, sclf_1_3, sclf_2_1 = None, None, None, None
lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1 = None, None, None, None, None, None, None
TRAIN_SAMPLE_WEIGHT = None
CACHEDIR = mkdtemp()

importing Jupyter notebook from My_Transformers.ipynb
importing Jupyter notebook from Word2Vec_Transformer.ipynb


2020-09-24 21:31:23,119 INFO adding document #0 to Dictionary(0 unique tokens: [])
2020-09-24 21:31:23,119 INFO built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [2]:
def pickleSave (obj, file):

    if VERSION != '':

        file, ext = os.path.splitext (file)
        file += "_v" + str (VERSION) + ext
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

def unpickle (file):

    if VERSION != '':

        file, ext = os.path.splitext(file)
        file += "_v" + str(VERSION) + ext
    return pickle.load (open (file, 'rb'))

In [3]:
class Discrete2OneHot_FeatureTransformer (BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        self.LB_1 = None
        self.LB_2 = None
        return

    def fit (self, X, y=None, **fit_params):
        print ('Discrete2OneHot_FeatureTransformer: fit(): X.shape =', X.shape)
        self.LB_1 = LabelBinarizer ()
        self.LB_1.fit (list (X['keyword']))
        self.LB_2 = LabelBinarizer ()
        self.LB_2.fit (list (pd.isnull (X['location'])))
        return self

    def transform (self, X, y=None, **fit_params):
        
        A = self.LB_1.transform (list (X['keyword']))
        B = self.LB_2.transform (list (pd.isnull (X['location']))).reshape (-1, 1)
        X = np.hstack ((A, B))
        print ('Discrete2OneHot_FeatureTransformer: transform(): X.shape =', X.shape)
        return X

# FV Trainers

In [4]:
# Global vars

tfidf_ft = Pipeline ([ 
    ('tfidf_text_location', ColumnTransformer ([ 
    
        ('tfidf_text', Pipeline ([

            ('tfidf_vectorizer',        TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)')),
            ('sparse_svm_feat_select1', SparseSVM_feature_selector ()),
            ('interactions',            PolynomialFeatures (2, interaction_only=True, include_bias=False)),
        ], memory=CACHEDIR), 'text'),
        ('tfidf_location',      TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)'), 'location'),
    ])), 
    ('standardization1',        StandardScaler (with_mean=False)),
    ('sparse_svm_feat_select2', SparseSVM_feature_selector ()),
    ('standardization2',        StandardScaler (with_mean=False)),
], memory=CACHEDIR)

pca_nmf_ft = Pipeline ([ 
    
    ('tfidf_ft',  tfidf_ft),
    ('pca_nmf',   PCA_NMF_TrainTest_FeatureTransformer (isSparseOut=True)),
], memory=CACHEDIR)

pca_category_ft = Pipeline ([
    ('pca_category', ColumnTransformer ([
    
        ('keyword_category', Discrete2OneHot_FeatureTransformer (), ['keyword', 'location']),  # LabelBinarizer (), ['keyword']),
        ('tfidf_pca',        Pipeline ([
            ('tfidf_ft',  tfidf_ft),
            ('pca',       PCA_NMF_TrainTest_FeatureTransformer (isNMF=False, isSparseOut=False))
        ], memory=CACHEDIR), ['text', 'location'])
    ])),
], memory=CACHEDIR)

# L-0 Classifiers

In [5]:
def create_fit_L0_clf (X, y):
    
    global tfidf_ft, pca_nmf_ft, pca_category_ft
    global lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1
    
    # Create
    lsvc_1 = Pipeline ([
        ('tfidf_ft',   tfidf_ft),
        ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
    ], memory=CACHEDIR)
    lsvc_1.name = 'lsvc_1'

    lsvc_2 = Pipeline ([
        ('pca_nmf_ft', pca_nmf_ft),
        ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
    ], memory=CACHEDIR)
    lsvc_2.name = 'lsvc_2'

    lsvc_3 = Pipeline ([
        ('pca_category_ft', pca_category_ft),
        ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
    ], memory=CACHEDIR)
    lsvc_3.name = 'lsvc_3'


    catb_1 = Pipeline ([
        ('tfidf_ft',   tfidf_ft),
        ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
    ], memory=CACHEDIR)
    catb_1.name = 'catb_1'

    catb_2 = Pipeline ([
        ('pca_nmf_ft', pca_nmf_ft),
        ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
    ], memory=CACHEDIR)
    catb_2.name = 'catb_2'

    catb_3 = Pipeline ([
        ('pca_category_ft', pca_category_ft),
        ('clf',             Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
    ], memory=CACHEDIR)
    catb_3.name = 'catb_3'

    fcnn_1 = Pipeline ([
        ('pca_category_ft', pca_category_ft),
        ('clf',             Best_clf_cv_transformer ({ 'name': 'FCNN', 'param_grid': {'max_epochs' : [20, 40]} }))
    ], memory=CACHEDIR)
    fcnn_1.name = 'fcnn_1'
    
    # fit
    lsvc_1.fit (X, y) 
    lsvc_2.fit (X, y) 
    lsvc_3.fit (X, y) 
    catb_1.fit (X, y)
    catb_2.fit (X, y)
    catb_3.fit (X, y)
    fcnn_1.fit (X, y)
    return

# Stacking Classifiers

In [6]:
def create_fit_stack_clf (X, y):
    
    global sclf_1_1, sclf_1_2, sclf_1_3, sclf_2_1
    global lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1
    
    # Level 1 create
    sclf_1_1 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1], 
                                   fit_base_estimators=False, use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
    sclf_1_1.name = 'sclf_1_1'

    sclf_1_2 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1], 
                                   fit_base_estimators=False, use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'RF',    'params': {'class_weight': 'balanced', 'n_estimators': 3, 'max_features': None} }))
    sclf_1_2.name = 'sclf_1_2'

    sclf_1_3 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1], 
                                   fit_base_estimators=False, use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False }))
    sclf_1_3.name = 'sclf_1_3'
    
    # fit
    sclf_1_1.fit (X, y)
    sclf_1_2.fit (X, y)
    sclf_1_3.fit (X, y)

    # Level 2 create
    sclf_2_1 = StackingClassifier (classifiers=[sclf_1_1, sclf_1_2, sclf_1_3], fit_base_estimators=False, 
                                   use_probas=True, average_probas=False, 
                                   meta_classifier=Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
    sclf_2_1.name = 'sclf_2_1'
    
    # fit
    sclf_2_1.fit (X, y)
    return    

# sample weights - don't know how to use this ? How to deal with unbalanced classes ?

In [7]:
def create_sample_weights ():
    
    global trainDF, TRAIN_SAMPLE_WEIGHT
    
    y = trainDF[pd.isnull (trainDF['target']) == False]['target']
    a = 1.0/ np.sum (y)
    b = 1.0/ (len (y) - np.sum (y))
    w1 = a/(a+b)
    w0 = b/(a+b)
    TRAIN_SAMPLE_WEIGHT = []
    for i in y:
        if i:
            TRAIN_SAMPLE_WEIGHT.append (w1)
        else:
            TRAIN_SAMPLE_WEIGHT.append (w0)
    return

In [8]:
def init ():
    
    global trainDF
    train_file = "Data/all.csv"
    trainDF = pd.read_csv (train_file)
    # trainDF = trainDF[0:20]
    trainDF['text'] = trainDF.text.astype (str)
    trainDF['keyword'] = trainDF.keyword.astype (str)
    trainDF['location'] = trainDF.location.astype (str)
    return

In [9]:
def fit ():
    
    create_fit_L0_clf (trainDF, trainDF['target'])
    create_fit_stack_clf (trainDF, trainDF['target'])
    return

In [10]:
def predict (clf, iterCount=0):
    
    print ("Starting Prediction:")
    test_file = "Data/test.csv"
    testDF = pd.read_csv (test_file)
    # testDF = testDF[0:10]
    testDF['text'] = testDF.text.astype (str)
    testDF['keyword'] = testDF.keyword.astype (str)
    testDF['location'] = testDF.location.astype (str)
    
    predictions = clf.predict (testDF)
    testDF['target'] = predictions
    testDF['target'] = testDF.target.astype (int)
    submitDF = testDF[['id', 'target']]
    submitDF.to_csv ('submission_v'+str (VERSION) + clf.name + '.' + str (iterCount+1) + '.csv', index=False)
    print ("Done Prediction !")
    return

In [11]:
# init ()
# fit ()
# predict (sclf_2_1)

In [12]:
def main (threshold=0.9, maxIterCount=1, thresh_depreciation=0.01):

    global trainDF, sclf_2_1
    init ()
    newTrainAddCount = 1
    iterCount = 0
    while newTrainAddCount > 0:

        threshold = threshold - iterCount * thresh_depreciation
        if iterCount >= maxIterCount:
            break
        fit ()
        predict (sclf_2_1, iterCount)
        predict (catb_1, iterCount)
        Pr = sclf_2_1.predict_proba (trainDF)
        newTrainAddCount = 0
        for idx, row in trainDF.iterrows ():
            if not pd.isnull (row['target']):

                continue
            print ('considering idx =', idx)
            pr = Pr[idx]
            p  = 0
            if pr[0] > 0.5:
                pr = pr[0]
                p  = 0
            else:
                pr = pr[1]
                p  = 1
            if pr >= threshold:
                # add in the training set
                trainDF.at[idx, 'target'] = p
                newTrainAddCount += 1
        # arrange df such that 1st rows with y=0/1 then rows with y=NaN
        arrangedDF1 = trainDF[pd.isnull (trainDF['target']) == False]
        arrangedDF2 = trainDF[pd.isnull (trainDF['target'])]
        trainDF = pd.concat ([arrangedDF1, arrangedDF2], axis=0)
        iterCount += 1
    trainDF.to_csv ('Data/all_'+str (VERSION) + sclf_2_1.name + '.' + str (iterCount+1)+'.csv', index=False)
    fit ()
    predict (sclf_2_1, iterCount)
    predict (catb_1, iterCount)
    
    # CV for the bclfs on the original train rows
    print ('------------------- CV for the sclf_2_1 on the original train rows -------------------')
    trainDF = trainDF[:TRAIN_ROW_COUNT]
    cv_results = []
    cv_result = cross_validate (sclf_2_1, trainDF, cv=5)
    cv_score = np.mean (cv_result['test_score'])
    print ()
    print (sclf_2_1.name, cv_score)
    print (cv_result)
    rmtree (CACHEDIR)
    return

In [ ]:
main (maxIterCount=1)

LinearSVC with L1-based feature selection for X.shape = (8234, 138022)
--------------------- Best parameter (CV score=0.699):
{'feature_selection__estimator__tol': 0.0001}
New #features =  1152
LinearSVC with L1-based feature selection for X.shape = (8234, 675795)
--------------------- Best parameter (CV score=0.675):
{'feature_selection__estimator__tol': 0.0001}
New #features =  5286
training LSvc for X.shape = (8234, 5286)
LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.933):
{'C': 0.001}
Done Fitting LSvc
Find optimal PCA dims and the same no. of NMF features for X.shape = (11497, 5286)
--------------------- Best:  parameters = (0.9, 0.1) , CV = 0.8507438662475545
PCA dimensionality, explainedVarRatio =  2647 0.9
training LSvc for X.shape = (8234, 5294)
LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.889):
{'C': 0.001}
Done Fitting LSvc
Discrete2OneHot_FeatureTransformer: fit(): X.shape = (11497, 2)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11

      2        0.0081       0.5865        0.0109  4.2060
      3        0.0056       0.5865        0.0087  4.0990
      4        0.0044       0.5865        0.0071  4.1240
      5        0.0036       0.5865        0.0062  4.0820
      6        0.0031       0.5865        0.0054  4.2060
      7        0.0027       0.5865        0.0050  4.1080
      8        0.0024       0.5865        0.0048  4.0560
      9        0.0022       0.5865        0.0040  4.2110
     10        0.0020       0.5865        0.0040  4.2020
     11        0.0019       0.5865        0.0037  4.1820
     12        0.0017       0.5865        0.0036  4.1700
     13        0.0016       0.5865        0.0033  4.0810
     14        0.0015       0.5865        0.0033  4.0930
     15        0.0014       0.5865        0.0030  4.3430
     16        0.0013       0.5865        0.0027  4.0530
     17        0.0013       0.5865        0.0026  4.2290
     18        0.0012       0.5865        0.0025  4.1100
     19        0.0011       0.5

     10        0.0021       0.5865        0.0034  4.1490
     11        0.0020       0.5865        0.0032  4.1940
     12        0.0018       0.5865        0.0030  4.2190
     13        0.0017       0.5865        0.0028  4.1630
     14        0.0016       0.5865        0.0026  4.1460
     15        0.0015       0.5865        0.0026  4.1620
     16        0.0014       0.5865        0.0025  4.2060
     17        0.0013       0.5865        0.0023  4.0830
     18        0.0013       0.5865        0.0022  4.2340
     19        0.0012       0.5865        0.0020  4.1790
     20        0.0011       0.5865        0.0020  4.1080
     21        0.0011       0.5865        0.0020  4.0160
     22        0.0010       0.5865        0.0019  4.1250
     23        0.0010       0.5865        0.0018  4.1260
     24        0.0009       0.5865        0.0017  4.0510
     25        0.0009       0.5865        0.0017  4.0520
     26        0.0009       0.5865        0.0017  4.1440
     27        0.0009       0.5

     39        0.0006       0.5865        0.0014  4.1470
     40        0.0006       0.5865        0.0015  4.1430
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.0310       0.5865        0.0144  5.1680
      2        0.0076       0.5865        0.0095  5.2190
      3        0.0052       0.5865        0.0076  5.1480
      4        0.0041       0.5865        0.0062  5.0700
      5        0.0033       0.5865        0.0054  5.1440
      6        0.0029       0.5865        0.0046  5.2350
      7        0.0025       0.5865        0.0043  5.1590
      8        0.0022       0.5865        0.0038  5.2270
      9        0.0020       0.5865        0.0036  5.0840
     10        0.0018       0.5865        0.0033  5.1520
     11        0.0017       0.5865        0.0030  5.0950
     12        0.0015       0.5865        0.0029  5.1840
     13        0.0014       0.5865        0.0028  5.1650
     14        0.0013       0.5

considering idx = 8911
considering idx = 8912
considering idx = 8913
considering idx = 8914
considering idx = 8915
considering idx = 8916
considering idx = 8917
considering idx = 8918
considering idx = 8919
considering idx = 8920
considering idx = 8921
considering idx = 8922
considering idx = 8923
considering idx = 8924
considering idx = 8925
considering idx = 8926
considering idx = 8927
considering idx = 8928
considering idx = 8929
considering idx = 8930
considering idx = 8931
considering idx = 8932
considering idx = 8933
considering idx = 8934
considering idx = 8935
considering idx = 8936
considering idx = 8937
considering idx = 8938
considering idx = 8939
considering idx = 8940
considering idx = 8941
considering idx = 8942
considering idx = 8943
considering idx = 8944
considering idx = 8945
considering idx = 8946
considering idx = 8947
considering idx = 8948
considering idx = 8949
considering idx = 8950
considering idx = 8951
considering idx = 8952
considering idx = 8953
considering

considering idx = 9411
considering idx = 9412
considering idx = 9413
considering idx = 9414
considering idx = 9415
considering idx = 9416
considering idx = 9417
considering idx = 9418
considering idx = 9419
considering idx = 9420
considering idx = 9421
considering idx = 9422
considering idx = 9423
considering idx = 9424
considering idx = 9425
considering idx = 9426
considering idx = 9427
considering idx = 9428
considering idx = 9429
considering idx = 9430
considering idx = 9431
considering idx = 9432
considering idx = 9433
considering idx = 9434
considering idx = 9435
considering idx = 9436
considering idx = 9437
considering idx = 9438
considering idx = 9439
considering idx = 9440
considering idx = 9441
considering idx = 9442
considering idx = 9443
considering idx = 9444
considering idx = 9445
considering idx = 9446
considering idx = 9447
considering idx = 9448
considering idx = 9449
considering idx = 9450
considering idx = 9451
considering idx = 9452
considering idx = 9453
considering

considering idx = 9911
considering idx = 9912
considering idx = 9913
considering idx = 9914
considering idx = 9915
considering idx = 9916
considering idx = 9917
considering idx = 9918
considering idx = 9919
considering idx = 9920
considering idx = 9921
considering idx = 9922
considering idx = 9923
considering idx = 9924
considering idx = 9925
considering idx = 9926
considering idx = 9927
considering idx = 9928
considering idx = 9929
considering idx = 9930
considering idx = 9931
considering idx = 9932
considering idx = 9933
considering idx = 9934
considering idx = 9935
considering idx = 9936
considering idx = 9937
considering idx = 9938
considering idx = 9939
considering idx = 9940
considering idx = 9941
considering idx = 9942
considering idx = 9943
considering idx = 9944
considering idx = 9945
considering idx = 9946
considering idx = 9947
considering idx = 9948
considering idx = 9949
considering idx = 9950
considering idx = 9951
considering idx = 9952
considering idx = 9953
considering

considering idx = 10410
considering idx = 10411
considering idx = 10412
considering idx = 10413
considering idx = 10414
considering idx = 10415
considering idx = 10416
considering idx = 10417
considering idx = 10418
considering idx = 10419
considering idx = 10420
considering idx = 10421
considering idx = 10422
considering idx = 10423
considering idx = 10424
considering idx = 10425
considering idx = 10426
considering idx = 10427
considering idx = 10428
considering idx = 10429
considering idx = 10430
considering idx = 10431
considering idx = 10432
considering idx = 10433
considering idx = 10434
considering idx = 10435
considering idx = 10436
considering idx = 10437
considering idx = 10438
considering idx = 10439
considering idx = 10440
considering idx = 10441
considering idx = 10442
considering idx = 10443
considering idx = 10444
considering idx = 10445
considering idx = 10446
considering idx = 10447
considering idx = 10448
considering idx = 10449
considering idx = 10450
considering idx 

considering idx = 10910
considering idx = 10911
considering idx = 10912
considering idx = 10913
considering idx = 10914
considering idx = 10915
considering idx = 10916
considering idx = 10917
considering idx = 10918
considering idx = 10919
considering idx = 10920
considering idx = 10921
considering idx = 10922
considering idx = 10923
considering idx = 10924
considering idx = 10925
considering idx = 10926
considering idx = 10927
considering idx = 10928
considering idx = 10929
considering idx = 10930
considering idx = 10931
considering idx = 10932
considering idx = 10933
considering idx = 10934
considering idx = 10935
considering idx = 10936
considering idx = 10937
considering idx = 10938
considering idx = 10939
considering idx = 10940
considering idx = 10941
considering idx = 10942
considering idx = 10943
considering idx = 10944
considering idx = 10945
considering idx = 10946
considering idx = 10947
considering idx = 10948
considering idx = 10949
considering idx = 10950
considering idx 

considering idx = 11410
considering idx = 11411
considering idx = 11412
considering idx = 11413
considering idx = 11414
considering idx = 11415
considering idx = 11416
considering idx = 11417
considering idx = 11418
considering idx = 11419
considering idx = 11420
considering idx = 11421
considering idx = 11422
considering idx = 11423
considering idx = 11424
considering idx = 11425
considering idx = 11426
considering idx = 11427
considering idx = 11428
considering idx = 11429
considering idx = 11430
considering idx = 11431
considering idx = 11432
considering idx = 11433
considering idx = 11434
considering idx = 11435
considering idx = 11436
considering idx = 11437
considering idx = 11438
considering idx = 11439
considering idx = 11440
considering idx = 11441
considering idx = 11442
considering idx = 11443
considering idx = 11444
considering idx = 11445
considering idx = 11446
considering idx = 11447
considering idx = 11448
considering idx = 11449
considering idx = 11450
considering idx 

In [ ]:
# clean file
def cleanDF ():
    
    import re
    text_cleaner_transformer = Text_cleaner_transformer ()
    files = ["Data/all.csv", "Data/train.csv", "Data/test.csv"]
    for file in files:
        
        df = pd.read_csv (file)
        df['text'] = text_cleaner_transformer.transform (df['text'])
        df.to_csv (file, index=False)
    return

In [ ]:
def null_count ():
    
    global trainDF
    
    init ()
    # get total count of data including missing data
    total = trainDF.isnull ().sum ().sort_values (ascending=False)
    # print (total)

    # get percent of missing data relevant to all data
    percent = (trainDF.isnull ().sum ()/trainDF.isnull ().count ()).sort_values (ascending=False)
    # print (percent)
    
    missing_data = pd.concat ([total, percent], axis=1, keys=['total', 'percent'])
    return missing_data    

In [ ]:
null_count ()

# Findings

